# Crime rate and Funding visualized

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#params
FP_SAFETY_DATA = os.path.join('data','safety_funding.csv')
FP_FIG_SPENDING_VS_CRIME_OUT = os.path.join('plots','spending_vs_crime.svg')
FP_FIG_SPENDING_OUT = os.path.join('plots','spending.svg')
assert os.path.exists(FP_SAFETY_DATA)
## NPD list of comparable cities
comp = ['Brookline', 
        'Cambridge',
        'Framingham',
        'Malden',
        'Medford',
        'Melrose',
        'Quincy',
        'Somerville',
        'Waltham',
        'Watertown']

In [ ]:
df = pd.read_csv(FP_SAFETY_DATA,delimiter='\t')
df['Police Budget per capita'] = df['Police Budget per capita'].str.replace('$','').astype(float)
newton = df[df.name=='Newton']
df = df.loc[:23].drop(19).infer_objects().sort_index(ascending=False).reset_index(drop=True)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
x = 'Total Crime Rate (per 1,000 residents)'
y = 'Police Budget per capita'
sns.regplot(data=df,
            x=x,
            y=y)
plt.scatter(newton[x],
            newton[y],
            marker='X',
            color='k',
            label='Newton')
plt.title('Crime and Police Funding in Greater Boston')
plt.legend(bbox_to_anchor = [1.35,1.0])
plt.savefig(FP_FIG_SPENDING_VS_CRIME_OUT)



In [ ]:
pdspending = 'Police Budget per capita'
crime = 'Total Crime Rate (per 1,000 residents)'

In [ ]:
(newton[pdspending] - df[pdspending].median())*int(newton.Population)

In [ ]:
%matplotlib inline
plt.figure(figsize=[3.5,5])
plt.hlines(df[pdspending].median(),-.1,0.1,label='median')
plt.scatter([0],newton[pdspending],marker='d',s=20,label='Newton')
sns.scatterplot(x=np.full(len(df),0),y=pdspending,data=df,hue='name',palette='BuGn_d')
plt.legend(bbox_to_anchor=[1.8,1])
plt.xlim(-0.5,0.5)
plt.tight_layout()
plt.savefig(FP_FIG_SPENDING_OUT)

# Field Interrogations and Observations
2015- May 2020


In [ ]:
# Params
FP_FIO = 'data/FIO.csv'
FP_DEMOGRAPHICS = 'data/newton_demographics.csv'
FP_FIG_FIO_COUNT_OUT = 'plots/fio_allstops_proportional_to_race.svg'

assert os.path.exists(FP_FIO)
assert os.path.exists(FP_DEMOGRAPHICS)

#demographic map
demo_map = {'W':'White',
            'B':'Black or African American',
            'A':'Asian',
            'I':'American Indian and Alaska Native',
            'H':'Hispanic or Latino (of any race)',
            'U':'unknown'}
map_demo_label = lambda demo_abbrev: demo_map[demo_abbrev]

In [ ]:
fio = pd.read_csv(FP_FIO)
fio.head()

In [ ]:
demo

In [ ]:
demo = pd.read_csv(FP_DEMOGRAPHICS,usecols=['demographic','Newton'])
#aggregate demographics not listed in FIOs as "unknown"
unknown_demos = ['Native Hawaiian and Other Pacific Islander','Some other race','Two or more races']
unknown_mask = [demographic in unknown_demos for demographic in demo.demographic]
demo.loc[8,:] = ['unknown',demo[unknown_mask].Newton.sum()]
demo = demo.set_index('demographic').drop(labels=unknown_demos).reset_index()
demo.columns=['demographic','demo_proportion']
demo

In [ ]:
fio_byrace = fio.groupby('Race').count()['Date/Time'].reset_index()
fio_byrace['Race'] = fio_byrace.Race.apply(map_demo_label)
fio_byrace.columns=['demographic','count']
fio_byrace['count_proportion'] = fio_byrace['count']/len(fio)

fio_byrace = fio_byrace.set_index('demographic').join(demo.set_index('demographic'))

#calculate FIO stops proportionally
count_proportionally = lambda x : x[1]/x[2]
fio_byrace['stop_liklihood'] = fio_byrace.apply(count_proportionally,raw=True,axis=1)

In [ ]:
fio_byrace

In [ ]:
%matplotlib inline
fio_byrace.stop_liklihood.plot(kind='bar')
plt.title('Liklihood of having police field interactions\nproportional to race/ethnicity')
plt.savefig(FP_FIG_FIO_COUNT_OUT)

In [ ]:
ratio_blktowhite = fio_byrace.loc['Black or African American'].stop_liklihood / fio_byrace.loc['White'].stop_liklihood
print(f'If you are Black in Newton, you are {np.round(ratio_blktowhite,3)} times more likely to have field interactions with the police than a white person')

In [ ]:
ratio_lattowhite = fio_byrace.loc['Hispanic or Latino (of any race)'].stop_liklihood / fio_byrace.loc['White'].stop_liklihood
print(f'If you are Latinx in Newton, you are {np.round(ratio_lattowhite,3)} times more likely to have field interactions with the police than a white person')

### What about officer initiated??

# Traffic stops

In [ ]:
#Params
FP_TRAFFIC_DATA = 'data/NPD race and traffic stops.csv'
FP_FIG_CITATIONS_OUT = 'plots/citation_rate.svg'
assert os.path.exists(FP_TRAFFIC_DATA)

In [ ]:
traffic = pd.read_csv(FP_TRAFFIC_DATA)
traffic.head()

In [ ]:
traffic_black = traffic.groupby('Race/ethnicity').get_group('Black')
traffic_black.Warnings.sum()/traffic_black.Citations.sum()

In [ ]:
traffic_white = traffic.groupby('Race/ethnicity').get_group('White')
traffic_white.Warnings.sum()/traffic_white.Citations.sum()

In [ ]:
races = traffic['Race/ethnicity'].unique()
by_race = traffic.groupby('Race/ethnicity')
warnings = [by_race.get_group(race).Warnings.sum() for race in races]
citations = [by_race.get_group(race).Citations.sum() for race in races]
demo_proportion = []
for race in races:
    demo_mask = [race in demographic for demographic in demo.demographic]
    prop = demo.demo_proportion[demo_mask].values
    demo_proportion.append(prop[0])
traffic_proportionally = pd.DataFrame({'demographic':races,
                                       'warnings':warnings,
                                       'citations':citations,
                                       'demo_prop':demo_proportion})
traffic_proportionally['warn_prop'] = traffic_proportionally.warnings/traffic_proportionally.warnings.sum()
traffic_proportionally['cit_prop'] = traffic_proportionally.citations/traffic_proportionally.citations.sum()

In [ ]:
traffic_proportionally

In [ ]:
#calculate traffic stop outcomes proportionally

citations_proportional_demo = lambda x : x[5]/x[3]
traffic_proportionally['citation_rate'] = traffic_proportionally.apply(citations_proportional_demo,raw=True,axis=1)

warnings_proportional_demo = lambda x : x[4]/x[3]
traffic_proportionally['warning_rate'] = traffic_proportionally.apply(warnings_proportional_demo,raw=True,axis=1)

In [ ]:
traffic_proportionally

In [ ]:
traffic_proportionally.set_index('demographic').drop('unknown').citation_rate.plot(kind='bar')
plt.savefig(FP_FIG_CITATIONS_OUT)

In [ ]:
likelihood_black = traffic_proportionally.set_index('demographic').loc['Black','citation_rate']/traffic_proportionally.set_index('demographic').loc['White','citation_rate']
print(f"If you're Black and driving in Newton you are {np.round(likelihood_black,3)} times more likely to get a traffic citation than if you're white")